In [61]:
%%capture --no-stderr
%pip install -U langgraph langsmith langchain_google_genai


In [62]:
import os
from google.colab import userdata

os.environ["LANGCHAIN_API_KEY"] = userdata.get('LANGCHAIN_API_KEY')
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "quickstart"

gemini_api_key = userdata.get('GEMINI_API_KEY')

In [63]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    max_retries=2,
    api_key=gemini_api_key
)

llm.invoke("greet me")


AIMessage(content='Hello there!\n', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-915e5109-fd79-436a-877b-bff1ab812475-0', usage_metadata={'input_tokens': 3, 'output_tokens': 4, 'total_tokens': 7, 'input_token_details': {'cache_read': 0}})

In [64]:
%%capture --no-stderr
%pip install -U tavily-python langchain_community


In [65]:
os.environ["TAVILY_API_KEY"] = userdata.get("TAVILY_API_KEY")

In [69]:
%pip install -U langsmith LangSmithLogger


ERROR: Could not find a version that satisfies the requirement LangSmithLogger (from versions: none)
ERROR: No matching distribution found for LangSmithLogger


In [69]:
%%capture --no-stderr
%pip install -U langchain langsmith langchain_community


In [77]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    max_retries=2,
    temperature=0.7,
    max_tokens=200,
    api_key=gemini_api_key
)

llm.invoke("greet me")


AIMessage(content='Hello there!\n', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-95dd8f03-2119-48af-9753-bdf54193653c-0', usage_metadata={'input_tokens': 3, 'output_tokens': 4, 'total_tokens': 7, 'input_token_details': {'cache_read': 0}})

In [82]:
import sqlite3
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.prompts.chat import ChatPromptTemplate
from langchain.chains import LLMChain
from langchain.llms import OpenAI
from langchain.tools import Tool
from langchain.agents import initialize_agent, AgentType
from langchain.memory import ConversationBufferMemory
import os

# Step 1: Set up API keys
os.environ["GEMINI_API_KEY"] = userdata.get('GEMINI_API_KEY')
os.environ["TAVILY_API_KEY"] = userdata.get("TAVILY_API_KEY")
os.environ["LANGCHAIN_API_KEY"] = userdata.get('LANGCHAIN_API_KEY')
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "quickstart"


# Step 2: Set up SQLite Database
def init_db():
    conn = sqlite3.connect("chatbot_data.db")
    cursor = conn.cursor()
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS chat_history (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            user_input TEXT NOT NULL,
            bot_response TEXT NOT NULL,
            timestamp DATETIME DEFAULT CURRENT_TIMESTAMP
        )
    """)
    conn.commit()
    conn.close()

def save_to_db(user_input, bot_response):
    conn = sqlite3.connect("chatbot_data.db")
    cursor = conn.cursor()
    cursor.execute("INSERT INTO chat_history (user_input, bot_response) VALUES (?, ?)", (user_input, bot_response))
    conn.commit()
    conn.close()

def fetch_history():
    conn = sqlite3.connect("chatbot_data.db")
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM chat_history ORDER BY timestamp DESC LIMIT 10")
    rows = cursor.fetchall()
    conn.close()
    return rows

# New: Function to delete conversation by ID
def delete_from_db(conversation_id):
    conn = sqlite3.connect("chatbot_data.db")
    cursor = conn.cursor()

    # First, check if the conversation exists
    cursor.execute("SELECT * FROM chat_history WHERE id = ?", (conversation_id,))
    row = cursor.fetchone()
    if row:
        # Proceed with deletion if found
        cursor.execute("DELETE FROM chat_history WHERE id = ?", (conversation_id,))
        conn.commit()
        print(f"Conversation with ID {conversation_id} has been deleted.")
    else:
        print(f"No conversation found with ID {conversation_id}.")

    conn.close()


# Step 3: Set up Tavily search tool
tavily_tool = TavilySearchResults(max_results=3)

# Step 4: Define the conversation memory
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# Step 5: Define the LLM
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    max_retries=2,
    temperature=0.7,
    max_tokens=200,
    api_key=gemini_api_key
)

# Step 6: Initialize tools
tools = [
    Tool(
        name="TavilySearch",
        func=tavily_tool.run,
        description="Useful for answering questions requiring research or external data."
    )
]

# Step 7: Create the conversational agent
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    memory=memory,
)
# Step 8: Implement the chatbot function with delete feature
def chatbot():
    print("Welcome to the Chatbot! Type 'exit' to end the conversation.\n")
    print("Type 'history' to view the last 10 conversations.\n")
    print("Type 'delete <id>' to delete a conversation by ID.\n")
    while True:
        user_input = input("You: ")
        if user_input.lower() in ["quit", "exit", "q"]:
            print("Goodbye!")
            break
        elif user_input.lower() in ["history", "data", "h"]:
            history = fetch_history()
            if not history:
                print("No history found.")
            else:
                for idx, (id, user, bot, timestamp) in enumerate(history, 1):
                    print(f"{idx}. {timestamp} - You: {user} | Bot: {bot}")
        elif user_input.lower().startswith("delete"):
            try:
                # Extract conversation ID to delete
                parts = user_input.split()
                if len(parts) > 1:
                    conversation_id = int(parts[1])
                    delete_from_db(conversation_id)  # Delete the conversation
                else:
                    print("Please provide a conversation ID to delete.")
            except ValueError:
                print("Invalid conversation ID. Please provide a numeric ID.")
        else:
            response = agent.run(user_input)
            print(f"Bot: {response}")

            # Save to SQLite database
            save_to_db(user_input, response)

# Step 9: Initialize and run the chatbot
if __name__ == "__main__":
    init_db()
    chatbot()




Welcome to the Chatbot! Type 'exit' to end the conversation.

Type 'history' to view the last 10 conversations.

Type 'delete <id>' to delete a conversation by ID.

You: h
1. 2024-12-22 00:31:23 - You: 671255be-8db3-4d4b-9667-f1c8e020cac0 | Bot: I cannot find any information associated with the UUID "671255be-8db3-4d4b-9667-f1c8e020cac0" using the available tools.  More context is needed.
2. 2024-12-22 00:21:45 - You: Remember my name? | Bot: I don't remember your name. Please tell me what it is.
3. 2024-12-22 00:21:24 - You: Hi there! My name is Junaid. | Bot: Hi Junaid!  It's nice to meet you.
4. 2024-12-22 00:20:21 - You: What's a 'node' in LangGraph? | Bot: In LangGraph, a node represents a function or computation step.  Each node performs a specific task, such as processing input or executing a tool.  Nodes can represent LLM agents, with edges defining communication channels between them.  They interact by reading and writing to a shared state.
5. 2024-12-22 00:19:37 - You: hi | B

In [ ]:
import sqlite3
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.prompts.chat import ChatPromptTemplate
from langchain.chains import LLMChain
from langchain.llms import OpenAI
from langchain.tools import Tool
from langchain.agents import initialize_agent, Tool, AgentType
from langchain.memory import ConversationBufferMemory


In [59]:
# Step 1: Set up SQLite Database
def init_db():
    conn = sqlite3.connect("chatbot_data.db")
    cursor = conn.cursor()
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS chat_history (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            user_input TEXT NOT NULL,
            bot_response TEXT NOT NULL,
            timestamp DATETIME DEFAULT CURRENT_TIMESTAMP
        )
    """)
    conn.commit()
    conn.close()

def save_to_db(user_input, bot_response):
    conn = sqlite3.connect("chatbot_data.db")
    cursor = conn.cursor()
    cursor.execute("INSERT INTO chat_history (user_input, bot_response) VALUES (?, ?)", (user_input, bot_response))
    conn.commit()
    conn.close()

def fetch_history():
    conn = sqlite3.connect("chatbot_data.db")
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM chat_history ORDER BY timestamp DESC LIMIT 10")
    rows = cursor.fetchall()
    conn.close()
    return rows

# Step 2: Set up Tavily search tool
tavily_tool = TavilySearchResults(max_results=3)

# Step 3: Define the conversation memory
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# Step 4: Define the LLM


# Step 5: Initialize tools
tools = [
    Tool(
        name="TavilySearch",
        func=tavily_tool.run,
        description="Useful for answering questions requiring research or external data."
    )
]

# Step 6: Create the conversational agent
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    memory=memory,
)

# Step 7: Implement the chatbot function
def chatbot():
    print("Welcome to the Chatbot! Type 'exit' to end the conversation.\n")
    print("Type 'history' to view the last 10 conversations.\n")
    while True:
        user_input = input("You: ")
        if user_input.lower() in ["quit", "exit", "q"]:
            print("Goodbye!")
            break
        elif user_input.lower() == "history":
            history = fetch_history()
            if not history:
                print("No history found.")
            else:
                for idx, (id, user, bot, timestamp) in enumerate(history, 1):
                    print(f"{idx}. {timestamp} - You: {user} | Bot: {bot}")
        else:
            response = agent.run(user_input)
            print(f"Bot: {response}")
            save_to_db(user_input, response)

# Step 8: Initialize and run the chatbot
if __name__ == "__main__":
    init_db()
    chatbot()


Welcome to the Chatbot! Type 'exit' to end the conversation.

Type 'history' to view the last 10 conversations.

You: q
Goodbye!
